In [ ]:
!git clone https://github.com/Mlawrence95/LinkedIn-Tech-Job-Data.git

In [ ]:
!pip install -U pip setuptools wheel
!pip install spacy-transformers
#!pip install pytextrank
!python -m spacy download en_core_web_trf
!python -m spacy download en_core_web_md
!python -m spacy download en_core_web_sm


In [1]:
import pandas as pd
import numpy as np
import csv
from utils import extract_wikidata, util_jobs, cleaner_jobs

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\39333\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\39333\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\39333\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\39333\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [2]:
df = pd.read_csv('datasets/LinkedIn-Tech-Job-Data/jobs.csv')
records = df.to_dict(orient='records')

In [ ]:
jobs_info = [ 'post_id', 'industries','job_function', 'title', 'abstract', 'post_url', 'entity_info_title', 'entity_info_abstract']

#new_dict = []
djob = {}
wikidata_ids = []
with open('datasets/LinkedIn-Tech-Job-Data/jobs.csv', 'a', encoding='utf-8', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames = jobs_info)
    #writer.writeheader()
    for job in records: ###
        djob["post_id"] = job['post_id']
        djob["industries"] = job['Industries']
        djob["job_function"] = job['Job function']
        djob["title"] = job['title']
        djob["abstract"] = extract_wikidata.extract_significant_sentences(job['description']) ##
        djob["post_url"] = job['post_url']
        #print((job['description']))
        #doc_trf = nlp_trf(job['description'])
        djob["entity_info_title"] = extract_wikidata.get_extract(job['description'], wikidata_ids)
        #print(djob["entity_info_title"])
        djob["entity_info_abstract"] = [] #remember to remove from util.py in for entity in eval(entity_info_abstract)
        #print(djob)
        writer.writerow(djob)



In [ ]:
file = open('datasets/LinkedIn-Tech-Job-Data/wikidata_ids.txt', 'w', encoding='utf-8')
for wikidata in wikidata_ids:
	file.write(wikidata + "\n")
file.close()

In [14]:
import numpy as np
with open('datasets/LinkedIn-Tech-Job-Data/jobs.csv', 'r', encoding='utf-8') as csv_file:
    # Load the data from the CSV file into a pandas DataFrame
    df = pd.read_csv(csv_file)

    # Calculate the number of rows in the DataFrame
    n = df.shape[0]

    # Generate a random permutation of the indices
    idx = np.random.permutation(n)

    # Split the indices into training and validation sets
    split = int(0.8 * n)
    train_idx = idx[:split]
    val_idx = idx[split:]

    # Split the DataFrame into training and validation sets based on the indices
    train_df = df.iloc[train_idx, :]
    val_df = df.iloc[val_idx, :]

    # Save the training and validation sets as CSV files
    train_df.to_csv('datasets/LinkedIn-Tech-Job-Data/train_jobs.tsv',  sep='\t', index=False)
    val_df.to_csv('datasets/LinkedIn-Tech-Job-Data/valid_jobs.tsv',  sep='\t', index=False)

In [8]:
import numpy as np
import pandas as pd
with open('datasets/LinkedIn-Tech-Job-Data/behaviors_jobs.tsv', 'r', encoding='utf-8') as csv_file:
# Load the data from the CSV file into a pandas DataFrame
    df = pd.read_csv(csv_file, sep='\t')
    df = df.drop(df.columns[:1], axis=1)

    # Calculate the number of rows in the DataFrame
    n = df.shape[0]

    # Generate a random permutation of the indices
    idx = np.random.permutation(n)

    # Split the indices into training and validation sets
    split = int(0.8 * n)
    train_idx = idx[:split]
    val_idx = idx[split:]

    # Split the DataFrame into training and validation sets based on the indices
    train_df = df.iloc[train_idx, :]
    val_df = df.iloc[val_idx, :]

    # Save the training and validation sets as CSV files
    train_df.to_csv('datasets/LinkedIn-Tech-Job-Data/behaviors_train_jobs.tsv',  sep='\t', index=False)
    val_df.to_csv('datasets/LinkedIn-Tech-Job-Data/behaviors_valid_jobs.tsv',  sep='\t', index=False)

In [2]:
# Config Generation
import time
import torch
import os
import argparse
from parse_config import ConfigParser
from utils import *

parser = argparse.ArgumentParser(description='KRED')
parser.add_argument('-f')
parser.add_argument('-c', '--config', default="./config.json", type=str,
                    help='config file path (default: None)')
parser.add_argument('-r', '--resume', default=None, type=str,
                    help='path to latest checkpoint (default: None)')
parser.add_argument('-d', '--device', default=None, type=str,
                    help='indices of GPUs to enable (default: all)')

config = ConfigParser.from_args(parser)

In [3]:
from utils.cleaner_jobs import *

config = cleaner_jobs(config)

In [4]:
from utils.util_jobs import entities_jobs
ent_jobs = entities_jobs(config)

In [5]:
from utils.util_jobs import entity_to_id_jobs

entity2id_dict = entity_to_id_jobs(config, ent_jobs)

In [6]:
from utils.util_jobs import construct_embedding_jobs, construct_adj_mind_jobs, id_to_entity_jobs
import numpy as np
entity_embedding = []
entity_embedding.append(np.zeros(config['model']['entity_embedding_dim']))
entity2embedding_dict = {}
entity2embedding_dict, entity_embedding, relation_embedding = construct_embedding_jobs(config, entity2id_dict, entity_embedding, entity2embedding_dict)

constructing embedding jobs...


In [7]:
from utils.util_jobs import construct_adj_mind_jobs
entity_adj, relation_adj = construct_adj_mind_jobs(config, entity2id_dict, entity2embedding_dict)


constructing adjacency matrix ...
construct_adj_mind finish


In [8]:
#tmp
for index in range(len(entity_adj)):
    if len(entity_adj[index]) == 0:
        entity_adj[index] = [0 for i in range(config['model']['entity_neighbor_num'])]
for index in range(len(relation_adj)):
    if len(relation_adj[index]) == 0:
        relation_adj[index] = [0 for i in range(config['model']['entity_neighbor_num'])]

In [9]:
from utils.util_jobs import id_to_entity_jobs
entities_not_embedded = set([item for items in entity_adj for item in items]).difference(set(entity2id_dict.values()))
entity2id_dict_not_embedded = id_to_entity_jobs(config, entities_not_embedded)
entity2embedding_dict, entity_embedding, relation_embedding = construct_embedding_jobs(config, entity2id_dict_not_embedded, entity_embedding, entity2embedding_dict)

# Add the new entities to the dictionary
entity2id_dict.update(entity2id_dict_not_embedded)
# Invert the dictionary
id2entity_dict = {v: k for k, v in entity2id_dict.items()}

constructing embedding jobs...


In [12]:
id2entity_dict

{1: 'Q192581',
 2: 'Q87',
 3: 'Q1370',
 4: 'Q47607',
 5: 'Q911679',
 6: 'Q82231',
 7: 'Q2539',
 8: 'Q61',
 9: 'Q28865',
 10: 'Q1276130',
 11: 'Q456157',
 12: 'Q17054505',
 13: 'Q708',
 14: 'Q7590',
 15: 'Q1191002',
 16: 'Q709113',
 17: 'Q21198',
 18: 'Q282049',
 19: 'Q145',
 20: 'Q20992346',
 21: 'Q8171',
 22: 'Q150225',
 23: 'Q98864861',
 24: 'Q30',
 25: 'Q10225',
 26: 'Q3733763',
 27: 'Q1158485',
 28: 'Q55168529',
 29: 'Q1726',
 30: 'Q183',
 31: 'Q51672715',
 32: 'Q19824629',
 33: 'Q1709010',
 34: 'Q5264328',
 35: 'Q516445',
 36: 'Q28390',
 37: 'Q428691',
 38: 'Q43035',
 39: 'Q11438',
 40: 'Q46794',
 41: 'Q8811',
 42: 'Q515838',
 43: 'Q2005',
 44: 'Q2849803',
 45: 'Q19399674',
 46: 'Q161053',
 47: 'Q850',
 48: 'Q47146',
 49: 'Q201904',
 50: 'Q63101604',
 51: 'Q18716',
 52: 'Q828721',
 53: 'Q2283',
 54: 'Q61526852',
 55: 'Q46744',
 56: 'Q112683502',
 57: 'Q251',
 58: 'Q1785501',
 59: 'Q172491',
 60: 'Q3353185',
 61: 'Q858810',
 62: 'Q175263',
 63: 'Q872685',
 64: 'Q19597236',
 65: 'Q3

In [10]:
id2entity_dict[0] = 'Q87'

In [11]:
for i in range(1, len(entity_adj)):
    for j in range(0, len(entity_adj[i])):
        #print(entity_adj[i][j])
        entity_adj[i][j] = entity2embedding_dict[id2entity_dict[entity_adj[i][j]]]
entity_embedding = torch.FloatTensor(np.array(entity_embedding))
relation_embedding = torch.FloatTensor(np.array(relation_embedding))

KeyError: 'Q919191'

In [23]:
entity_embedding.shape

torch.Size([13356, 100])

In [24]:
#rivedi train e valid jobs + verifica correttezza per ogni colonna (no \t)

In [25]:
from utils.util_jobs import build_news_features_mind_jobs

embedding_folder=None
jobs_feature, max_entity_freq, max_entity_pos, max_entity_type = build_news_features_mind_jobs(config, entity2embedding_dict, embedding_folder)



In [35]:
# Load the user history
import importlib
importlib.reload(util_jobs)
from utils.util_jobs import build_user_history_jobs
user_history = build_user_history_jobs(config) #manca train_behaviour

In [36]:
user_history

{'user_train': ['Histories',
  'N0',
  'N0',
  'N0',
  'N0',
  'N0',
  'N0',
  'N0',
  'N0',
  'N0',
  'N0',
  'N0',
  'N0',
  'N0',
  'N0',
  'N0',
  'N0',
  'N0',
  'N0',
  'N0'],
 'U9283_train': ['J4318',
  'J1960',
  'J1739',
  'N0',
  'N0',
  'N0',
  'N0',
  'N0',
  'N0',
  'N0',
  'N0',
  'N0',
  'N0',
  'N0',
  'N0',
  'N0',
  'N0',
  'N0',
  'N0',
  'N0'],
 'U6235_train': ['J2907',
  'J1405',
  'J6104',
  'J6600',
  'J1353',
  'J2719',
  'J5286',
  'N0',
  'N0',
  'N0',
  'N0',
  'N0',
  'N0',
  'N0',
  'N0',
  'N0',
  'N0',
  'N0',
  'N0',
  'N0'],
 'U7824_train': ['J1286',
  'J437',
  'J142',
  'J5096',
  'J2740',
  'J2894',
  'J4301',
  'J2508',
  'N0',
  'N0',
  'N0',
  'N0',
  'N0',
  'N0',
  'N0',
  'N0',
  'N0',
  'N0',
  'N0',
  'N0'],
 'U7072_train': ['J7151',
  'J4896',
  'J7998',
  'J6705',
  'J5439',
  'J3091',
  'J3482',
  'J2888',
  'J5298',
  'J1694',
  'N0',
  'N0',
  'N0',
  'N0',
  'N0',
  'N0',
  'N0',
  'N0',
  'N0',
  'N0'],
 'U1975_train': ['J8253',
  'J30

In [37]:
from utils.util_jobs import get_user2item_data_jobs

train_data, dev_data = get_user2item_data_jobs(config)


In [33]:
import importlib
from utils import util_jobs
from utils import cleaner_jobs, extract_wikidata
from model import KGAT
importlib.reload(extract_wikidata)
importlib.reload(util_jobs)
importlib.reload(cleaner_jobs)
importlib.reload(KGAT)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\39333\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\39333\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\39333\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\39333\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


<module 'model.KGAT' from 'J:\\Downloads\\NLP\\kred\\model\\KGAT.py'>

In [38]:
from utils.util_jobs import build_vert_data_jobs
vert_train, vert_test = build_vert_data_jobs(config)


{'Information Technology and Services, Computer Software, and Internet': 0, 'Computer Software, Consumer Services, and Information Technology and Services': 1, 'Information Technology and Services, Computer Software, and Financial Services': 2, nan: 3, 'Entertainment': 4, 'Information Technology and Services': 5, 'Information Technology and Services, Defense & Space, and Computer Software': 6, 'Internet': 7, 'Consumer Electronics': 8, 'Computer Software': 9, 'Marketing and Advertising, Information Technology and Services, and Computer Software': 10, 'Information Technology and Services, Consumer Electronics, and Marketing and Advertising': 11, 'Financial Services': 12, 'Internet and Information Technology and Services': 13, 'Information Services, Financial Services, and Banking': 14, 'Pharmaceuticals': 15, 'Electrical/Electronic Manufacturing and Computer Software': 16, 'Computer Hardware': 17, 'Construction, Retail, and Hospital & Health Care': 18, 'Staffing and Recruiting': 19, 'Comp

In [39]:
from utils.util_jobs import build_pop_data_jobs
pop_train, pop_test = build_pop_data_jobs(config)


In [40]:
from utils.util_jobs import build_item2item_data_jobs
item2item_train, item2item_test = build_item2item_data_jobs(config)

In [41]:
import gzip
import pickle
data_dict = {
    'user_history' : user_history,
    'entity_embedding' : entity_embedding,
    'relation_embedding' : relation_embedding,
    'entity_adj' : entity_adj,
    'relation_adj' : relation_adj,
    'jobs_feature': jobs_feature,
    'max_entity_freq':max_entity_freq,
    'max_entity_pos': max_entity_pos,
    'max_entity_type':max_entity_type,
    'train_data': train_data,
    'dev_data': dev_data,
    'vert_train': vert_train,
    'vert_test' : vert_test,
    'pop_train': pop_train,
    'pop_test':pop_test,
    'item2item_train': item2item_train,
    'item2item_test': item2item_test
}

save_compressed_pickle(f'/datasets/LinkedIn-Tech-Job-Data/data_dict_jobs.pkl', data_dict)

In [ ]:
import pandas as pd
import csv
from utils import extract_wikidata, util_jobs

In [17]:
import pandas as pd
import csv
from utils import extract_wikidata
user_info = ["User", "Entities"]

df = pd.read_csv('datasets/LinkedIn-Tech-Job-Data/clean_stackoverflow_survey.csv')
#new_dict = []
djob = {}
with open('datasets/LinkedIn-Tech-Job-Data/user_entities_stackoverflow_tmp.csv', 'w', encoding='utf-8', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames = user_info)
    writer.writeheader()

    for i, job in zip(range(0, 10), df):
        wikidata_ids = []
        djob["User"] = "U"+ str(i)
        language = extract_wikidata.get_extract(df.Language[i], wikidata_ids)
        database = extract_wikidata.get_extract(df.Database[i], wikidata_ids)
        platform = extract_wikidata.get_extract(df.Platform[i], wikidata_ids)
        webframe = extract_wikidata.get_extract(df.Webframe[i], wikidata_ids)
        misctech = extract_wikidata.get_extract(df.MiscTech[i], wikidata_ids)
        toolstech = extract_wikidata.get_extract(df.ToolsTech[i], wikidata_ids)
        opsys = extract_wikidata.get_extract(df.OpSys[i], wikidata_ids)

        #print(djob["User"], language, database, platform, webframe, misctech, toolstech, opsys)
        djob["Entities"] = wikidata_ids

        writer.writerow(djob)


In [21]:
djob

{'User': 'U9',
 'Entities': ['Q2283',
  'Q1785501',
  'Q2350702',
  'Q725967',
  'Q28925578',
  'Q2225',
  'Q2852503',
  'Q1537445',
  'Q28957072']}